In [1]:
import numpy as np
import tensorflow as tf

from keras.preprocessing.sequence import pad_sequences
import pickle

Using TensorFlow backend.


In [2]:
import data_utils
import embedding_utils
import display_utils

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
np.random.seed(1001)
tf.set_random_seed(1001)

### Load data

In [5]:
VOCAB_SIZE  = 50000
#load dataset
with open('aux_files/dataset_%d.pkl' %VOCAB_SIZE, 'rb') as f:
    dataset = pickle.load(f)

In [6]:

#load conter-fitted word 
dist_mat = np.load('aux_files/dist_counter_%d.npy' %VOCAB_SIZE)
# Prevent returning 0 as most similar word because it is not part of the dictionary
dist_mat[0,:] = 100000
dist_mat[:,0] = 100000

skip_list = np.load('aux_files/missed_embeddings_counter_%d.npy' %VOCAB_SIZE)


In [7]:
doc_len = [len(dataset.val_X[i]) for i in 
           range(len(dataset.val_X))]

### Demonstrating how we find the most similar words

In [8]:
src_word = dataset.dict['好']
src_word
dataset.inv_full_dict[50000]

'伟'

In [9]:
for i in range(6230, 6232):
    src_word = i
    nearest, nearest_dist = embedding_utils.pick_most_similar_words(src_word, dist_mat,20, 0.5)
        
    print('Closest to `%s` are:' %(dataset.inv_dict[src_word]))
    for w_id, w_dist in zip(nearest, nearest_dist):
          print(' -- ', dataset.inv_dict[w_id], ' ', w_dist)

    print('----')

Closest to `国税局` are:
----
Closest to `翼` are:
 --  机翼   0.22788355864099952
 --  辅   0.22916311992500127
 --  翅膀   0.2738878111520009
 --  援   0.3341176668279999
 --  辅助   0.34568997297100035
 --  副   0.3563555582999989
 --  扶持   0.36099929984099965
 --  救援   0.3625574274400005
 --  支架   0.3828246505230002
 --  帮   0.390361236298
 --  援助   0.3941633762969998
 --  养   0.4035929243710008
 --  救   0.4049656322590014
 --  架   0.417898789983
 --  赞助   0.42399925209200195
 --  扶   0.42721877353100135
 --  资助   0.43061200651000053
 --  助益   0.4325585738910007
 --  声援   0.43391956708499957
 --  托   0.43443959343899996
----


### Loading the NLI model

In [10]:
from NLImodel.models import NLImodelClass


In [11]:
model = NLImodelClass(is_train=False)


Learning Rate:  0.0007
Model Loaded.


In [12]:
dataset.val_X[0], dataset.val_Y[0], dataset.val_Z[0], np.argmax(dataset.val_Z[0])

(array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
          328, 50000,    29,  2923,  1991,   236,   108, 50000,  2631,
           29,   106,  2348, 50000,  1099,    29,  2106,    15,    52,
          620,    62, 50000], dtype=int32),
 array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,    29,   900,   262,    15,    52,
         3915,  1670, 50000], dtype=int32),
 array([0., 0., 1.]),
 2)

In [13]:
np.argmax(model.predict(dataset.val_X[0], dataset.val_X[0])) 

1

## Try Attack

In [14]:
from attacks import GreedyAttack, GeneticAtack

### Main Attack 

In [15]:
pop_size = 32
n1 = 8
goog_lm = None

ga_atttack = GeneticAtack(model, model, model, dataset, dist_mat, 
                                  skip_list,
                                  goog_lm, max_iters=30, 
                                   pop_size=pop_size,
                                  
                                  n1 = n1,
                                  n2 = 4,
                                 use_lm = False, use_suffix=False)

In [16]:
len(dataset.val_Z)

5010

In [23]:
TEST_SIZE = len(dataset.val_Z)#1000
#use val data for attack
#test_idx = np.random.choice(len(dataset.val_Z), TEST_SIZE, replace=False)#随机index,总共1000个
test_idx = [x for x in range(len(dataset.val_Z))]
test_len = []
for i in range(TEST_SIZE):
    test_len.append(len(dataset.val_Y[test_idx[i]]))
print('Shortest sentence in our test set is %d words' %np.min(test_len))

test_list = []
orig_list = []
orig_list_no_max = []
orig_label_list = []
adv_list = []
dist_list = []

for i in range(TEST_SIZE):
    print('Index:',i)
    x_orig = [dataset.val_X[test_idx[i]], dataset.val_Y[test_idx[i]]]
    x_orig_no_max = [dataset.val_X_nomax[test_idx[i]], dataset.val_Y_nomax[test_idx[i]]]
    orig_label = np.argmax(dataset.val_Z[test_idx[i]])
    orig_preds=  model.predict(x_orig[0], x_orig[1])
    # print(orig_label, orig_preds, np.argmax(orig_preds))
    if np.argmax(orig_preds) != orig_label:
        print('skipping wrong classifed ..')
        print('--------------------------')
        continue
    x_len = np.sum(np.sign(x_orig[1]))
    '''
    if x_len >= 30:
        print('skipping too long input..')
        print('--------------------------')
        continue
    '''
    # if np.max(orig_preds) < 0.90:
    #    print('skipping low confidence .. \n-----\n')
    #    continue
    print('****** ', len(test_list) + 1, ' ********')
    test_list.append(test_idx[i])
    orig_list.append(x_orig)
    orig_list_no_max.append(x_orig_no_max)
    #target_label = 1 if orig_label == 2 else 2
    orig_label_list.append(orig_label)
    x_adv = ga_atttack.attack( x_orig, orig_label)
    adv_list.append(x_adv)
    if x_adv is None:
        print('%d failed' %(i+1))
        dist_list.append(100000)
    else:
        num_changes = np.sum(x_orig[1] != x_adv)
        print('%d - %d changed.' %(i+1, num_changes))
        dist_list.append(num_changes)
        # display_utils.visualize_attack(sess, model, dataset, x_orig, x_adv)
    print('--------------------------')
    #if (len(test_list)>= 50):
    #    break

Shortest sentence in our test set is 30 words
Index: 0
skipping wrong classifed ..
--------------------------
Index: 1
skipping wrong classifed ..
--------------------------
Index: 2
skipping wrong classifed ..
--------------------------
Index: 3
skipping wrong classifed ..
--------------------------
Index: 4
******  1  ********
		 0  --  0.33548075
		 1  --  0.35591453
		 2  --  0.46766135
5 - 1 changed.
--------------------------
Index: 5


/home/haejuu/aaai2019/haejuu/lib/python3.4/site-packages/ipykernel_launcher.py:50: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.


skipping wrong classifed ..
--------------------------
Index: 6
******  2  ********
		 0  --  0.01855124
		 1  --  0.06783889
		 2  --  0.17104943
		 3  --  0.17104943
		 4  --  0.2035459
		 5  --  0.23126893
		 6  --  0.27026942
		 7  --  0.27026942
		 8  --  0.27026942
		 9  --  0.27805752
		 10  --  0.27805752
		 11  --  0.27805752
		 12  --  0.27805752
		 13  --  0.27805752
		 14  --  0.27805752
		 15  --  0.27805752
		 16  --  0.27805752
		 17  --  0.27805752
		 18  --  0.27805752
		 19  --  0.27805752
		 20  --  0.27805752
		 21  --  0.2780575
		 22  --  0.2780575
		 23  --  0.2780575
		 24  --  0.2780575
		 25  --  0.2780575
		 26  --  0.2780575
		 27  --  0.2780575
		 28  --  0.2780575
		 29  --  0.2780575
		 0  --  0.13200133
		 1  --  0.21096495
		 2  --  0.31029806
		 3  --  0.43130267
		 4  --  0.58954537
7 - 1 changed.
--------------------------
Index: 7
******  3  ********
		 0  --  0.22963917
		 1  --  0.30928457
		 2  --  0.342826
8 - 1 changed.
------------------------

		 13  --  0.1903238
		 14  --  0.1903238
		 15  --  0.1903238
		 16  --  0.1903238
		 17  --  0.1903238
		 18  --  0.1903238
		 19  --  0.19032383
		 20  --  0.19032383
		 21  --  0.19032383
		 22  --  0.19032383
		 23  --  0.19032383
		 24  --  0.19032383
		 25  --  0.19032383
		 26  --  0.19032383
		 27  --  0.19032383
		 28  --  0.19032383
		 29  --  0.19032383
40 failed
--------------------------
Index: 40
******  27  ********
		 0  --  0.19487788
		 1  --  0.27102116
		 2  --  0.27102116
		 3  --  0.28005219
		 4  --  0.34365115
		 5  --  0.34365115
		 6  --  0.34365115
		 7  --  0.3522786
41 - 1 changed.
--------------------------
Index: 41
skipping wrong classifed ..
--------------------------
Index: 42
******  28  ********
		 0  --  0.33782035
		 1  --  0.48608026
43 - 1 changed.
--------------------------
Index: 43
******  29  ********
		 0  --  0.40401357
44 - 1 changed.
--------------------------
Index: 44
******  30  ********
		 0  --  0.23236728
		 1  --  0.30260205
		 2 

		 0  --  0.2932856
		 1  --  0.4215236
71 - 1 changed.
--------------------------
Index: 71
skipping wrong classifed ..
--------------------------
Index: 72
******  45  ********
		 0  --  0.55511475
73 - 1 changed.
--------------------------
Index: 73
skipping wrong classifed ..
--------------------------
Index: 74
******  46  ********
		 0  --  0.64823246
75 - 1 changed.
--------------------------
Index: 75
skipping wrong classifed ..
--------------------------
Index: 76
skipping wrong classifed ..
--------------------------
Index: 77
skipping wrong classifed ..
--------------------------
Index: 78
skipping wrong classifed ..
--------------------------
Index: 79
******  47  ********
		 0  --  0.37144387
		 1  --  0.39382923
80 - 1 changed.
--------------------------
Index: 80
******  48  ********
		 0  --  0.16025756
		 1  --  0.38698983
81 - 1 changed.
--------------------------
Index: 81
skipping wrong classifed ..
--------------------------
Index: 82
******  49  ********
		 0  -- 

...

		 5  --  0.4281452
1237 - 1 changed.
--------------------------
Index: 1237
skipping wrong classifed ..
--------------------------
Index: 1238
******  772  ********
		 0  --  0.35857877
		 1  --  0.7172937
1239 - 1 changed.
--------------------------
Index: 1239
skipping wrong classifed ..
--------------------------
Index: 1240
******  773  ********
		 0  --  0.30817622
		 1  --  0.49994224
1241 - 1 changed.
--------------------------
Index: 1241
******  774  ********
		 0  --  0.31162873
		 1  --  0.5116179
1242 - 1 changed.
--------------------------
Index: 1242
******  775  ********
		 0  --  0.37332842
1243 - 1 changed.
--------------------------
Index: 1243
******  776  ********
		 0  --  0.30982345
		 1  --  0.44728437
1244 - 1 changed.
--------------------------
Index: 1244
skipping wrong classifed ..
--------------------------
Index: 1245
skipping wrong classifed ..
--------------------------
Index: 1246
******  777  ********
		 0  --  0.31610245
		 1  --  0.37651545
1247 - 1 

		 0  --  0.36055523
		 1  --  0.5445917
1288 - 1 changed.
--------------------------
Index: 1288
******  803  ********
		 0  --  0.39283407
		 1  --  0.44589502
1289 - 1 changed.
--------------------------
Index: 1289
******  804  ********
		 0  --  0.47608808
1290 - 1 changed.
--------------------------
Index: 1290
******  805  ********
		 0  --  0.38199574
1291 - 1 changed.
--------------------------
Index: 1291
skipping wrong classifed ..
--------------------------
Index: 1292
skipping wrong classifed ..
--------------------------
Index: 1293
skipping wrong classifed ..
--------------------------
Index: 1294
******  806  ********
		 0  --  0.11675163
		 1  --  0.4979819
1295 - 1 changed.
--------------------------
Index: 1295
******  807  ********
		 0  --  0.22579934
		 1  --  0.3656397
		 2  --  0.3656397
		 3  --  0.3809636
		 4  --  0.3809636
		 5  --  0.4165302
1296 - 1 changed.
--------------------------
Index: 1296
******  808  ********
		 0  --  0.3502455
1297 - 1 changed.


		 20  --  0.36754957
		 21  --  0.36754957
		 22  --  0.36754957
		 23  --  0.36754957
		 24  --  0.36754957
		 25  --  0.36754957
		 26  --  0.36754957
		 27  --  0.36754957
		 28  --  0.36754957
		 29  --  0.36754957
		 0  --  0.30097765
		 1  --  0.34559524
		 2  --  0.34559524
		 3  --  0.34559527
		 4  --  0.34559527
		 5  --  0.34559527
		 6  --  0.34559527
		 7  --  0.34559527
		 8  --  0.34559527
		 9  --  0.34559527
		 10  --  0.34559527
		 11  --  0.34559527
		 12  --  0.3455953
		 13  --  0.3455953
		 14  --  0.3455953
		 15  --  0.3455953
		 16  --  0.3455953
		 17  --  0.3455953
		 18  --  0.3455953
		 19  --  0.3455953
		 20  --  0.3455953
		 21  --  0.3455953
		 22  --  0.3455953
		 23  --  0.3455953
		 24  --  0.3455953
		 25  --  0.3455953
		 26  --  0.3455953
		 27  --  0.3455953
		 28  --  0.3455953
		 29  --  0.3455953
1335 failed
--------------------------
Index: 1335
skipping wrong classifed ..
--------------------------
Index: 1336
skipping wrong classifed ..
--

		 0  --  0.1437915
		 1  --  0.2024526
		 2  --  0.23040767
		 3  --  0.30475277
		 4  --  0.43207482
1397 - 1 changed.
--------------------------
Index: 1397
******  864  ********
		 0  --  0.4179469
1398 - 1 changed.
--------------------------
Index: 1398
******  865  ********
		 0  --  0.33090103
		 1  --  0.5650872
1399 - 1 changed.
--------------------------
Index: 1399
******  866  ********
		 0  --  0.24930935
		 1  --  0.33932954
		 2  --  0.41649202
1400 - 1 changed.
--------------------------
Index: 1400
******  867  ********
		 0  --  0.33164418
		 1  --  0.47597063
1401 - 1 changed.
--------------------------
Index: 1401
******  868  ********
		 0  --  0.43078455
1402 - 1 changed.
--------------------------
Index: 1402
******  869  ********
		 0  --  0.2703237
		 1  --  0.52859175
1403 - 1 changed.
--------------------------
Index: 1403
******  870  ********
		 0  --  0.2400903
		 1  --  0.33382377
		 2  --  0.33382377
		 3  --  0.33510277
		 4  --  0.3351028
		 5  --  0.3

ValueError: Error when checking : expected input_2 to have shape (None, 30) but got array with shape (0, 1)

### Visualize some output results

In [24]:
print('Median number of modifications ' , np.median([x for x in dist_list if x != 100000]))

Median number of modifications  1.0


In [25]:
orig_len = [np.sum(np.sign(x)) for x in orig_list[1]]
ratio = [dist_list[i]/orig_len[i] for i in range(len(orig_list[1]))]

In [26]:
ratio_success = [x for x in ratio if x < 1.0]
np.mean(ratio_success)

0.07941176470588235

In [30]:
#visual_idx = 0
for visual_idx in range(len(adv_list)):
    print(visual_idx+1)
    display_utils.visualize_attack(model, dataset, orig_list_no_max[visual_idx], adv_list[visual_idx])

1
Original Prediction = entailment. (Confidence = 52.13) 
Premise


Hypothesis


---------  After attack -------------
New Prediction = contradiction. (Confidence = 46.65) 


2
Original Prediction = contradiction. (Confidence = 97.20) 
Premise


Hypothesis


...

Hypothesis


---------  After attack -------------
New Prediction = entailment. (Confidence = 47.21) 


879
Original Prediction = contradiction. (Confidence = 43.01) 
Premise


Hypothesis


---------  After attack -------------
New Prediction = entailment. (Confidence = 47.68) 
